In [1]:
import pandas as pd
import numpy as np



### Get all the pillar names from the excel

In [2]:
names = pd.read_excel('../../UNDP Digital Assessment Data Framework Filename Matching V7.xlsx')

In [3]:
col_names = ['Indicator','check', 'Data Source','Index','Filename']

In [4]:
names = names[col_names]

In [5]:
names.head()

,Indicator,check,Data Source,Index,Filename
0,Countries,NaN,United Nations,False,Countries
1,"Database of Global Administrative Areas (GADM,...",NaN,GADM maps and data,False,NaN
2,High Resolution Population Density Maps + Demo...,NaN,Facebook,False,NaN
3,population density vs openstreetmap object den...,NaN,Kontur,False,NaN
4,Population Density,Infrastructure,World Bank,False,population_density


In [6]:
# get all the files per pillar
data_stats = names.groupby('check').agg({'Filename':'count','Indicator':'count'})

In [7]:
data_stats

,Filename,Indicator
check,,
Business,20,25
Foundations,9,12
Government,10,15
Infrastructure,39,48
People,35,47
Regulation,6,7
Strategy,1,1


### Foundations

In [8]:
bnames = names[(names.check=='Foundations')&(~names.Filename.isna())]#&(names.Index==False)]

In [9]:
bnames.head(25)

,Indicator,check,Data Source,Index,Filename
148,Digital payments penetration,Foundations,Portulans Institute,TRUE,digital_payments_penetration
149,% of population with digital finance account -...,Foundations,World Bank,False,population_digital_financial_services
150,% of population with digital finance account -...,Foundations,World Bank,False,population_digital_financial_services
153,Identification For Development (ID4D) Global D...,Foundations,World Bank,False,id4d_nid
154,% of population with ID,Foundations,World Bank,False,id4d_nid
155,% of services that can be accessed,Foundations,World Bank,False,id4d_services
156,can ID be used for transactions,Foundations,World Bank,False,id4d_services
157,Is personal data siloed,Foundations,World Bank,False,Egov_strategy
158,Open data index,Foundations,Open Knowledge Foundation,True,open_data_idx


In [10]:
# get list of names for all indicators
indicators = bnames.Indicator.unique()

In [11]:
# get all file names
bfiles = bnames.Filename.unique()

In [12]:
bfiles

array(['digital_payments_penetration',
       'population_digital_financial_services', 'id4d_nid',
       'id4d_services', 'Egov_strategy', 'open_data_idx'], dtype=object)

In [13]:
# ls digital-readiness-assessment-main/processed/

In [14]:
##ict_goods and services not in process data

In [15]:
# formula for converting scale
def convert_rank(old_value, old_min=1, old_max=7, new_min=1, new_max=6 ):
    """ Convert old scale values scale into new scale values"""
    old_range = old_max - old_min
    new_range = new_max - new_min
    new_value = (((old_value-old_min)*new_range)/old_range)+new_min
    return new_value

### 1. Digital payments penetration

In [16]:
indicators[0]

'Digital payments penetration'

In [17]:
# load data
indicator = indicators[0]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Digital payments penetration
digital_payments_penetration


In [18]:
df.head()

,RANK,COUNTRY/ECONOMY,VALUE,SCORE
0,1.0,Norway,0.85,100.00
1,2.0,Denmark,0.83,97.24
2,3.0,Finland,0.80,93.95
3,4.0,Sweden,0.80,93.08
4,5.0,Netherlands,0.76,89.01


In [19]:
# score looks like the one to use
df.describe()

,RANK,VALUE,SCORE
count,122.000000,122.000000,122.000000
mean,61.500000,0.315738,36.076967
std,35.362409,0.209012,24.867406
min,1.000000,0.010000,0.000000
25%,31.250000,0.150000,16.235000
50%,61.500000,0.270000,30.870000
75%,91.750000,0.440000,51.110000
max,122.000000,0.850000,100.000000


In [20]:
# df.Indicator.unique()

In [21]:
# create standard columns
df.rename(columns={'COUNTRY/ECONOMY':'Country Name'}, inplace=True)
df['higher_is_better'] = True
df['Indicator'] = indicator
df['data_col'] = df['SCORE'] 


min_rank = df['data_col'].min()
max_rank = df['data_col'].max()

# transform 1-100 rank into 1-6
df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))



In [22]:
df

,RANK,Country Name,VALUE,SCORE,higher_is_better,Indicator,data_col,new_rank_score
0,1.0,Norway,0.85,100.00,True,Digital payments penetration,100.00,6.0000
1,2.0,Denmark,0.83,97.24,True,Digital payments penetration,97.24,5.8620
2,3.0,Finland,0.80,93.95,True,Digital payments penetration,93.95,5.6975
3,4.0,Sweden,0.80,93.08,True,Digital payments penetration,93.08,5.6540
4,5.0,Netherlands,0.76,89.01,True,Digital payments penetration,89.01,5.4505
...,...,...,...,...,...,...,...,...
129,NaN,Jamaica,NaN,NaN,True,Digital payments penetration,NaN,NaN
130,NaN,Madagascar,NaN,NaN,True,Digital payments penetration,NaN,NaN
131,NaN,Oman,NaN,NaN,True,Digital payments penetration,NaN,NaN
132,NaN,Qatar,NaN,NaN,True,Digital payments penetration,NaN,NaN


In [23]:

# output scores to csv
df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].to_csv('../indicator_scores/foundations_{}_scores.csv'.format(bf), index=False)

In [24]:
df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].head(15)


,Country Name,Indicator,data_col,new_rank_score,higher_is_better
0,Norway,Digital payments penetration,100.00,6.0000,True
1,Denmark,Digital payments penetration,97.24,5.8620,True
2,Finland,Digital payments penetration,93.95,5.6975,True
3,Sweden,Digital payments penetration,93.08,5.6540,True
4,Netherlands,Digital payments penetration,89.01,5.4505,True
5,New Zealand,Digital payments penetration,86.73,5.3365,True
6,United States,Digital payments penetration,84.40,5.2200,True
7,Estonia,Digital payments penetration,82.82,5.1410,True
8,"Korea, Rep.",Digital payments penetration,82.64,5.1320,True
9,Canada,Digital payments penetration,81.53,5.0765,True


## 2. % of population with digital finance account - registered


In [25]:
indicator = indicators[1]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population with digital finance account - registered
population_digital_financial_services


In [26]:
# no DATE associated wiht the data

In [27]:
df.columns.tolist()

['Unnamed: 0',
 'Account (% age 15+)',
 'Account, male (% age 15+)',
 'Account, in labor force (% age 15+) ',
 'Account, out of labor force (% age 15+) ',
 'Account, female (% age 15+)',
 'Account, young adults (% ages 15-24)',
 'Account, older adults (% ages 25+)',
 'Account, primary education or less (% ages 15+) ',
 'Account, secondary education or more (% ages 15+) ',
 'Account, income, poorest 40% (% ages 15+)',
 'Account, income, richest 60% (% ages 15+) ',
 'Account, rural (% age 15+) ',
 'Financial institution account (% age 15+) ',
 'Financial institution account,male(% age 15+) ',
 'Financial institution account, in labor force(% age 15+) ',
 'Financial institution account, out of labor force (% age 15+) ',
 'Financial institution account,female(% age 15+) ',
 'Financial institution account,young adults(% age 15-24) ',
 'Financial institution account, older adults(% age 25+) ',
 'Financial institution account, primary education or less(% age 15+) ',
 'Financial institution ac

In [28]:
df[['data_country',
 'data_year','Unnamed: 0']].drop_duplicates()

,data_country,data_year,Unnamed: 0
0,Low income,Low income,Low income
3,Lower middle income,Lower middle income,Lower middle income
5,Upper middle income,Upper middle income,Upper middle income
8,NaN,NaN,NaN
11,High income,High income,High income


In [29]:
df.head(16)

,Unnamed: 0,Account (% age 15+),"Account, male (% age 15+)","Account, in labor force (% age 15+)","Account, out of labor force (% age 15+)","Account, female (% age 15+)","Account, young adults (% ages 15-24)","Account, older adults (% ages 25+)","Account, primary education or less (% ages 15+)","Account, secondary education or more (% ages 15+)",...,"Mobile money account, female (% age 15+)","Mobile money account, young adults (% age 15-24)","Mobile money account, older adults (% age 25+)","Mobile money account, primary education or less (% age 15+)","Mobile money account, secondary education or less (% age 15+)","Mobile money account, income, poorest 40% (% age 15+)","Mobile money account, income, richest 60% (% age 15+)","Mobile money account, rural (% age 15+)",data_country,data_year
0,Low income,9%,15%,15%,2%,3%,6%,11%,5%,30%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low income,Low income
1,Low income,10%,16%,15%,4%,4%,7%,12%,5%,23%,...,0%,0%,0%,0%,0%,0%,1%,0%,Low income,Low income
2,Low income,15%,23%,25%,4%,7%,10%,18%,9%,31%,...,1%,0%,1%,0%,2%,0%,1%,1%,Low income,Low income
3,Lower middle income,39%,39%,46%,31%,39%,30%,45%,35%,44%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lower middle income,Lower middle income
4,Lower middle income,29%,36%,36%,12%,22%,15%,38%,14%,70%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lower middle income,Lower middle income
5,Upper middle income,28%,34%,36%,15%,23%,26%,29%,15%,36%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Upper middle income,Upper middle income
6,Upper middle income,38%,43%,44%,29%,34%,30%,40%,24%,56%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Upper middle income,Upper middle income
7,Upper middle income,40%,42%,53%,27%,38%,32%,43%,27%,56%,...,2%,6%,1%,1%,4%,0%,4%,2%,Upper middle income,Upper middle income
8,NaN,22%,30%,33%,11%,14%,15%,26%,14%,31%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,30%,38%,42%,18%,22%,21%,34%,21%,39%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# going to use the  score column since this is already an Index

In [31]:
# # convert rank into 1-6 - in order to get most recent data 
# df['data_col'] = df['Score'] 

# # going to assume index is between 1-100 but not 100% sure
# min_rank = 1 #df['data_col'].min()
# max_rank = 100 #df['data_col'].max()

# # transform 1-100 rank into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

# # need to invert score since higher rank is not better 
# # df['new_rank_score'] = (6-df['new_rank_score'])+1

In [32]:
# # prepare output
# df.rename(columns={'Country':'Country Name'}, inplace=True)
# df['Indicator'] = indicator
# df['higher_is_better'] = True

# df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].head(15)


# # output scores to csv
# df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].to_csv('../indicator_scores/{}_scores.csv'.format(bf), index=False)

## 3. % of population with digital finance account - active (90 days)

In [33]:
indicator = indicators[2]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population with digital finance account - active (90 days)
population_digital_financial_services


In [34]:
df.head()

,Unnamed: 0,Account (% age 15+),"Account, male (% age 15+)","Account, in labor force (% age 15+)","Account, out of labor force (% age 15+)","Account, female (% age 15+)","Account, young adults (% ages 15-24)","Account, older adults (% ages 25+)","Account, primary education or less (% ages 15+)","Account, secondary education or more (% ages 15+)",...,"Mobile money account, female (% age 15+)","Mobile money account, young adults (% age 15-24)","Mobile money account, older adults (% age 25+)","Mobile money account, primary education or less (% age 15+)","Mobile money account, secondary education or less (% age 15+)","Mobile money account, income, poorest 40% (% age 15+)","Mobile money account, income, richest 60% (% age 15+)","Mobile money account, rural (% age 15+)",data_country,data_year
0,Low income,9%,15%,15%,2%,3%,6%,11%,5%,30%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low income,Low income
1,Low income,10%,16%,15%,4%,4%,7%,12%,5%,23%,...,0%,0%,0%,0%,0%,0%,1%,0%,Low income,Low income
2,Low income,15%,23%,25%,4%,7%,10%,18%,9%,31%,...,1%,0%,1%,0%,2%,0%,1%,1%,Low income,Low income
3,Lower middle income,39%,39%,46%,31%,39%,30%,45%,35%,44%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lower middle income,Lower middle income
4,Lower middle income,29%,36%,36%,12%,22%,15%,38%,14%,70%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lower middle income,Lower middle income


No country or date data. Also not clear which column to use.

In [35]:
# dcol = 'Percentage of total trade in services'
# indicol = 'Category Label'
# cname = 'Economy Label'

# # filter most recent year and imports
# df = df[(df.Year==2019)&(df['Flow Label']=='Imports')]

# # create the standard columns
# df['higher_is_better'] = True
# df['Indicator'] =  df[indicol]+' '+ df['Flow Label']
# df['Country Name'] = df[cname]
# # df['2019 [YR2019]'] = df['2019 [YR2019]'].astype(float)
# df['data_col'] = df[dcol]


# # convert 1-100 %  into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# df.sort_values(by='new_rank_score', ascending=False)

# # prepare output
# df = df[['Country Name', 'Indicator', 'data_col', 'new_rank_score','higher_is_better']]

# # output scores
# df.to_csv('../indicator_scores/{}_scores.csv'.format(bf+'_imports'), index=False)

## 4. Identification For Development (ID4D) Global Dataset


In [36]:
indicator = indicators[3]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Identification For Development (ID4D) Global Dataset
id4d_nid


In [37]:
df.head()

,Registered Population Above Cut-Off Age,Unnamed: 1,Country Characteristics,NID age,Data Source Category,Registered Population,By Gender,By Rural/Urban,By Age Group,data_country,data_year
0,id,NaN,Region,Mandatory NID Age,Type,RPB (Registered Population Above Cut-off Age),RPB Male,RPB Rural,RPB 0-30,NaN,NaN
1,1,NaN,SAS,0,Voter,"20,845,988","13,549,892",NaN,NaN,NaN,NaN
2,2,NaN,ECS,16,Direct,"4,455,772","2,267,673",NaN,"1,863,436",NaN,NaN
3,3,NaN,MEA,18,Voter,"23,251,503",NaN,NaN,NaN,NaN,NaN
4,4,NaN,ECS,-,Voter,"24,512",NaN,NaN,NaN,NaN,NaN


In [38]:
df.columns

Index(['Registered Population Above Cut-Off Age', 'Unnamed: 1',
       'Country Characteristics', 'NID age', 'Data Source Category',
       'Registered Population', 'By Gender', 'By Rural/Urban', 'By Age Group',
       'data_country', 'data_year'],
      dtype='object')

In [39]:
df[['data_country', 'data_year']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   data_country  0 non-null      float64
 1   data_year     0 non-null      float64
dtypes: float64(2)
memory usage: 3.2 KB


In [40]:
# dcol = 'Percentage of total trade in services'
# indicol = 'Category Label'
# cname = 'Economy Label'

# # filter most recent year
# df = df[(df.Year==2019)&(df['Flow Label']=='Exports')]

# # create standard columns
# df['higher_is_better'] = True
# df['Indicator'] =  df[indicol]+' '+ df['Flow Label']
# df['Country Name'] = df[cname]
# # df['2019 [YR2019]'] = df['2019 [YR2019]'].astype(float)
# df['data_col'] = df[dcol]


# # convert 1-100 %  into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# df.sort_values(by='new_rank_score', ascending=False)

# # prepare output
# df = df[['Country Name', 'Indicator', 'data_col', 'new_rank_score','higher_is_better']]

# # output scores
# df.to_csv('../indicator_scores/{}_scores.csv'.format(bf+'_exports'), index=False)

## % of population with ID


In [41]:
indicator = indicators[4]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

% of population with ID
id4d_nid


In [42]:
df['Country Characteristics'].unique()

array(['Region', 'SAS', 'ECS', 'MEA', 'SSF', 'LCN', 'EAS', 'NAC'],
      dtype=object)

In [43]:
df.head()§

SyntaxError: invalid character in identifier (<ipython-input-43-4d81f80abdff>, line 1)

In [ ]:
# dcol = 'Percentage of total merchandise trade'
# indicol = 'IctProductCategory Label'
# cname = 'Economy Label'

# # filter most recent year and exports
# df = df[(df.Year==2019)&(df['Flow Label']=='Exports')]

# # create standard columns
# df['higher_is_better'] = True
# df['Indicator'] =  df[indicol]+' '+ df['Flow Label']
# df['Country Name'] = df[cname]
# # df['2019 [YR2019]'] = df['2019 [YR2019]'].astype(float)
# df['data_col'] = df[dcol]


# # convert 1-100 %  into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# df.sort_values(by='new_rank_score', ascending=False)

# # prepare output
# df = df[['Country Name', 'Indicator', 'data_col', 'new_rank_score','higher_is_better']]

# # output scores
# df.to_csv('../indicator_scores/{}_scores.csv'.format(bf+'_exports'), index=False)

## 6. % of services that can be accessed

In [ ]:
indicator = indicators[5]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

In [ ]:
df.info()

In [ ]:
df['Freedom on the Net (2017)'].unique()

In [ ]:
df.head()

In [ ]:
# dcol = 'Percentage of total merchandise trade'
# indicol = 'IctProductCategory Label'
# cname = 'Economy Label'

# # filter most recent year and exports
# df = df[(df.Year==2019)&(df['Flow Label']=='Imports')]

# # create standard columns
# df['higher_is_better'] = True
# df['Indicator'] =  df[indicol]+' '+ df['Flow Label']
# df['Country Name'] = df[cname]
# # df['2019 [YR2019]'] = df['2019 [YR2019]'].astype(float)
# df['data_col'] = df[dcol]


# # convert 1-100 %  into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row, old_min=0,old_max=100))

# df.sort_values(by='new_rank_score', ascending=False)

# # prepare output
# df = df[['Country Name', 'Indicator', 'data_col', 'new_rank_score','higher_is_better']]

# # output scores
# df.to_csv('../indicator_scores/{}_scores.csv'.format(bf+'_imports'), index=False)

## 7. can ID be used for transactions



In [ ]:
indicator = indicators[6]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

In [ ]:
df.head()

In [ ]:
# min_rank = 1
# max_rank = df['Country Name'].nunique()

# # transform 1-24 rank into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

# # need to invert score since higher rank is not better 
# # df['new_rank_score'] = (6-df['new_rank_score'])+1

In [ ]:
# # prepare output
# df.sort_values(by='new_rank_score', ascending=False)

# # prepare output
# df = df[['Country Name', 'Indicator', 'data_col', 'new_rank_score','higher_is_better']]

# # output scores
# df.to_csv('../indicator_scores/{}_scores.csv'.format(bf), index=False)

## 8. Is personal data siloed


In [ ]:
indicator = indicators[7]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

In [ ]:
df.head()

In [ ]:
# # filter on relevant years
# df = df[(df['Time']==2017)&(df['Information and communication technologies']=='ICT-intensive')]

# # convert rank into 1-6 - in order to get most recent data 
# df['data_col'] = df['Value'] 

# # going to assume index is between 1-100 but not 100% sure
# min_rank = 0 #df['data_col'].min()
# max_rank = 100 #df['data_col'].max()

# # transform 1-147 rank into 1-6
# df['new_rank_score'] = df['data_col'].apply(lambda row: convert_rank(row,old_min=min_rank,old_max=max_rank))

# # need to invert score since higher rank is not better 
# # df['new_rank_score'] = (6-df['new_rank_score'])+1

# # df_rank[['Country ISO3', 'Country Name','Indicator','data_col','new_rank_score']].head()

# df.rename(columns={'Country':'Country Name'}, inplace=True)
# df['Indicator'] = indicator
# df['higher_is_better'] = True

# df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].head(15)


# # output scores to csv
# df[['Country Name','Indicator','data_col','new_rank_score','higher_is_better']].to_csv('../indicator_scores/{}_scores.csv'.format(bf), index=False)

## 9. Open data index

In [ ]:
bnames

In [44]:
indicator = indicators[8]
print(indicator)
bf = bnames[bnames['Indicator']==indicator]['Filename'].values[0]
print(bf)

df = pd.read_csv('../../processed/{}.csv'.format(bf))

Open data index
open_data_idx


FileNotFoundError: [Errno 2] No such file or directory: '../../processed/open_data_idx.csv'

### Score Aggregating

In [ ]:
import os


In [ ]:
# get list of files in scores folder
scores = os.listdir('../indicator_scores/')
scores = [s for s in scores if s.startswith('foundations')]

In [ ]:
scores

In [ ]:
# create a dataframe that concatenates all these file into one table
df = pd.concat([pd.read_csv('../indicator_scores/{}'.format(s)) for s in scores])    

In [ ]:
df

In [ ]:
# Data cleaning
df['new_rank_score'] = df['new_rank_score'].fillna(0)
df.sort_values(by=['Country Name'], ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.info()

In [ ]:
df.head(15)

In [ ]:
df.describe()

In [ ]:
# checking country names
sorted(df['Country Name'].unique().tolist())

In [ ]:
# remove trailing whitespaces from country name
df['Country Name'] = df['Country Name'].str.strip()


In [ ]:
# checking country names
sorted(df['Country Name'].unique().tolist())

In [ ]:
# average indicator scores per country
agg_df = df.groupby(['Country Name']).agg({'new_rank_score':'mean','data_col':'count'})

In [ ]:
agg_df.columns = ['agg_score', 'count_source' ]

In [ ]:
max_number_sources = agg_df.describe()['count_source']['max']

In [ ]:
agg_df['agg_score_wt'] = agg_df['agg_score']*(agg_df['count_source']/max_number_sources)

In [ ]:
agg_df.sort_values(by='agg_score', ascending=False, inplace=True)

In [ ]:
agg_df.head(25)

In [ ]:
agg_df.to_csv('../pillar_scores/foundation_scores_v0.csv')